## Real-Time Testing Code
Run the following code to evaluate the model in real-time. When activated, the model will identify the gestures and display the corresponding text on the screen upon recognition. Users have the option to test the model using any of the 15 gestures demonstrated in the tutorial video.

In [8]:
#SetUp
import cv2
import mediapipe as mp
from matplotlib import pyplot as plt

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [9]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [10]:
def draw_landmarks(image, results):
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 ) 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(128, 0, 128), thickness=2, circle_radius=2)
                                 ) 
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0, 0, 128), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 ) 
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0, 0, 128), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 ) 

In [11]:
import numpy as np

def extract_keypoints(results):
    if results.pose_landmarks:  
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    else:  
        pose = np.zeros(33*4)

    if results.face_landmarks:  
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
    else: 
        face = np.zeros(468*3)

    if results.left_hand_landmarks:  
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else:  
        lh = np.zeros(21*3)

    if results.right_hand_landmarks:  
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    else: 
        rh = np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [12]:
greeting_actions = ['Hello', 'Goodbye', 'Please', 'Sorry', 'Welcome']
basic_actions = ['Maybe', 'Family', 'Should', 'Yes', 'Sure']
question_actions = ['Ask','Why', 'What', 'When', 'How']

actions = greeting_actions + basic_actions + question_actions

In [ ]:
import tensorflow as tf
import pyttsx3
import time

video = []
sentence = []
threshold = 0.8
cap = cv2.VideoCapture(0)

# Users can test one of the three models by removing the comment symbol from the chosen model's code. 
model = tf.keras.models.load_model('grubslmodel.keras')
# model = tf.keras.models.load_model('simplernn.keras')
# model = tf.keras.models.load_model('lstmmodel.keras')
with mp_holistic.Holistic (
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        success, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        keypoints = extract_keypoints(results)
        video.insert(0,keypoints)
        video = video[:30]
        if len(video) == 30:
            res = model.predict(np.expand_dims(video, axis=0))[0]
            print('Detected Word:' + actions[np.argmax(res)])
            if res[np.argmax(res)] > threshold:
                predicted_action = actions[np.argmax(res)]
                if len(sentence) <= 0 or predicted_action != sentence[-1]:
                    sentence.append(predicted_action)
            if len(sentence) > 1: 
                sentence = sentence[-1:]
            
        cv2.rectangle(image, (0,0), (640, 40), (200, 200, 200), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0,0,0), 1, cv2.LINE_AA)

        cv2.imshow('Real-Time Testing', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [14]:
cap.release()
cv2.destroyAllWindows()